In [28]:
import numpy as np
from scipy import optimize
import sympy as sm
import matplotlib.pyplot as plt



# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
from Funcs import *
from Model import *
model = production_economy()
model.setup()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Question 1
We first find the market clearing conditions, i.e. the value of $l^\star$, for the different prices defined as $p_1$ in `linspace(0.1,2.0,10)` and $p_2$ in `linspace(0.1,2.0,10)`.

In [29]:
# Define the linspaces
p1_value = np.linspace(0.1, 2.0, 10)
p2_value = np.linspace(0.1, 2.0, 10)

# Empty array to store results
results = []

for p1 in p1_value:
    for p2 in p2_value:
        if model.market_clearing(p1, p2) != None:
            labor, good1, good2 = model.market_clearing(p1, p2)
            results.append([p1, p2, labor, good1, good2])

            # Print the results
            for result in results:
                p1, p2, labor, good1, good2 = result
                print(f"p1: {p1:.2f}, p2: {p2:.2f}, Labor MC: {labor:.4f}, Good1 MC: {good1:.4f}, Good2 MC: {good2:.4f}")

# Print number of markets clearing
print(f"Number of markets not clearing: {len(p1_value)*len(p2_value) - len(results)}")
print(f"Number of markets clearing: {len(results)}")


Prices of good 1: 0.1
Prices of good 2: 0.1
Markets does not clear
Prices of good 1: 0.1
Prices of good 2: 0.3111111111111111
Markets does not clear
Prices of good 1: 0.1
Prices of good 2: 0.5222222222222223
Markets does not clear
Prices of good 1: 0.1
Prices of good 2: 0.7333333333333333
Markets does not clear
Prices of good 1: 0.1
Prices of good 2: 0.9444444444444444
Markets does not clear
Prices of good 1: 0.1
Prices of good 2: 1.1555555555555557
Markets does not clear
Prices of good 1: 0.1
Prices of good 2: 1.3666666666666667
Markets does not clear
Prices of good 1: 0.1
Prices of good 2: 1.577777777777778
Markets does not clear
Prices of good 1: 0.1
Prices of good 2: 1.788888888888889
Markets does not clear
Prices of good 1: 0.1
Prices of good 2: 2.0
Markets does not clear
Prices of good 1: 0.3111111111111111
Prices of good 2: 0.1
Markets does not clear
Prices of good 1: 0.3111111111111111
Prices of good 2: 0.3111111111111111
Markets does not clear
Prices of good 1: 0.3111111111111

# Question 2
We now want to find the equilibrium prices p1 and p2 that ensures market clearing. 
We will use Walras' law such that by having 2 markets lcear, the clearing of market 3 is ensured.

In [30]:
initial_guess = [model.par.p1, model.par.p2]

equilibrium_prices = optimize.fsolve(Walras_law, initial_guess)

# Print the results
p1_equilibrium, p2_equilibrium = equilibrium_prices
par.p1 = p1_equilibrium
par.p2 = p2_equilibrium
print(f"Equilibrium prices: p1 = {p1_equilibrium:.4f}, p2 = {p2_equilibrium:.4f}")

# Calculate the equilibrium values
laborD_eq = labor_demand(model.par, p1_equilibrium) + labor_demand(model.par, p2_equilibrium)
good1_eq, good2_eq = demand(model.par, laborD_eq, p1_equilibrium, p2_equilibrium)
y1_eq = production(model.par, p1_equilibrium)
y2_eq = production(model.par, p2_equilibrium)

# Print the results
print(f"Equilibrium values: Labor demand = {laborD_eq:.4f}, Good1 = {good1_eq:.4f}, Good2 = {good2_eq:.4f}, Y1 = {y1_eq:.4f}, Y2 = {y2_eq:.4f}")

Equilibrium prices: p1 = 1.5826, p2 = 2.4174
Equilibrium values: Labor demand = 2.0871, Good1 = 0.7913, Good2 = 1.2087, Y1 = 0.7913, Y2 = 1.2087


We see that both goods market clears, and thus according to Walras law we know that the market for labor supply and demand must also clear.  
The solutions are thus:  
$c_1=y_1=0.7913$  
$c_2=y_2=1.2087$  
$\ell^\star=2.0871$

# Question 3
The government now cares about the social welfare function:
$$SWF=U-\kappa y_2^\star$$
We now want to find the value of tax, $\tau$, and lump sum transfers, $T$, that maximizes this function.  
We are told that the governmen balances its budget such that $T=\tau c_2^\star$.

In [35]:
# Outer loop: Maximizing the social welfare function
p1 = model.par.p1
p2 = model.par.p2
l = labor_demand(model.par, p1)+labor_demand(model.par, p2)
c1 , c2 = demand(model.par, l, p1, p2)

# Imposing the government's budget constraint
constraint = ({'type': 'ineq', 'fun': lambda tau: p1*c1 + (p2 + tau)*c2 - (par.w*l + tau*c2 + profit(model.par, p1) + profit(model.par, p2))})

# Initial guess
initial_guess = 0.5

# Optimize
res = optimize.minimize(SWF, initial_guess, args=(model.par), constraints=constraint)

# Extract the optimal tax
tau_optimal = res.x
T_optimal = tau_optimal*c2

# Print the results
print(f"Optimal tax: {tau_optimal[0]:.4f}")



TypeError: SWF() takes 1 positional argument but 2 were given